In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta

In [12]:
%run ../interesting_options.py
%run my_3dbar_plotter.py

In [13]:
%who_ls

['add_z_score',
 'calculate_horizon_date',
 'calls1',
 'calls2',
 'calls3',
 'calls4',
 'date',
 'datetime',
 'datetime_valid',
 'df_combiner',
 'df_to_3Dbars',
 'filter_by_date',
 'filter_by_z_score',
 'interesting_options',
 'max_interest_by',
 'my_3dbar_plotter',
 'np',
 'o_chains_to_3d_df',
 'parse_contractSymbol',
 'pd',
 'plotter',
 'plt',
 'timedelta']

In [6]:
# load calls option chains
calls1, calls2, calls3 = (
    pd.read_csv('D:\\code\\test\\csv\\pltr\\calls1.csv', index_col=0),
    pd.read_csv('D:\\code\\test\\csv\\pltr\\calls2.csv', index_col=0),
    pd.read_csv('D:\\code\\test\\csv\\pltr\\calls3.csv', index_col=0),
)
print('calls1', calls1.shape)
print('calls2', calls2.shape)
print('calls3', calls3.shape)

calls1 (436, 20)
calls2 (387, 20)
calls3 (391, 20)


In [7]:
calls1, calls2, calls3 = (
    interesting_options(calls1, time_horizon='6mo'),
    interesting_options(calls2, time_horizon='6mo'),
    interesting_options(calls3, time_horizon='6mo'),
)
print('calls1', calls1.shape)
print('calls2', calls2.shape)
print('calls3', calls3.shape)

calls1 (78, 23)
calls2 (81, 23)
calls3 (87, 23)


In [37]:
# my_3dbar_plotter accepts a misdf
def o_chains_to_3d_df(o_chains, x='date_strike', y='columns', z='openInterest', colsasdays=True):
    """
    Takes a number of option chains (each == one timeperiod of observation, default = 1 day).
    Returns a df where: 
        x = index (expiration date for MIS, or contract symbol) - x-axis
        y = column (observation time period, day1, day2 etc) - y-axis
        z = whatever we are looking at (open interest for MIS) - z-axis
        
    By default we count number of o_chains and create colums for each o_chain: colsasdays=True
    """
    
    # TODO - else y is dodgy
    columns = np.arange(0, len(o_chains), 1) if colsasdays is True else y 
    
    # I need to combine all o_chains indexes to make enough room in 3D bar plot
    # slice each chain
    def slice_chains(chains, x, y, z):
        out = []
        for i, chain in enumerate(chains):
#             print(chain['exp'])
            out.append(pd.DataFrame(chain[z].values, columns=[i], index=chain['exp']))
        return out
   
    sliced = slice_chains(o_chains, x, y, z)
    # print(sliced) # ok
    out_df = df_combiner(sliced)
#     print(out_df)
#     out_df = pd.DataFrame([], columns=columns, index=index)
    
    return out_df
    
# I need to combine columns

# df = pd.DataFrame(opin_calls['openInterest'].values, index=opin_calls['date_strike'])
# df1 = pd.DataFrame(opin_calls1['openInterest'].values, index=opin_calls1['date_strike'])
# df.columns = ['df']
# df1.columns = ['df1']
# df2 = df.combine_first(df1)

In [41]:
# now plot a 3D bar chart
# x = expirations
# y = days of observation
# z = open_interest
mis_df = o_chains_to_3d_df([calls1, calls2, calls3])
# mis_df.to_csv('D:\\code\\test\\csv\\pltr\\mis_df.csv') # damn df_combiner botches up
# my_3dbar_plotter(mis_df)